Resquisitos:

Criar um linkedin fake

Instalar o selenium

Instalar o webdriver Manager

pip install selenium

pip install webdriver-manager

In [1]:
from selenium import webdriver #Biblioteca de automação
from selenium.webdriver.common.by import By #Função utilizada pra achar os componentes html 
from webdriver_manager.firefox import GeckoDriverManager #Biblioteca específica para quem vai utilizar o navegador firefox
from selenium.webdriver.firefox.service import Service #Identifica a versão do seu navegador e para o selenium funcionar corretamente
from selenium.webdriver.common.keys import Keys #Função do selenium que automatiza a entrada de caracteres
from bs4 import BeautifulSoup # Biblioteca que faz a coleta dos dados
from time import sleep # Usado para deixar o programa o mais humano possível evitando o banimento de ip
import csv # usado para gravar os resultados do scrape

In [2]:
#Passo 01 Instalando configurando o webdriver do firefox

att = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=att)

print('- Passo 01 concluído.')

[WDM] - Downloading: 16.2kB [00:00, 16.6MB/s]                   


In [ ]:
#Passo 1.1: Abrindo a página de login no linkedin

#Definindo a URL
url = 'https://www.linkedin.com/login' 

#Usando o navegador o Firefox para acessar a url com método driver.get()
driver.get(url)
sleep(3)

print('- Passo 1.1 concluído.')

Você deve criar um arquivo em txt contendo o email se senha do linkedin fake

In [8]:
# 1.2 Buscando o arquivo no sistema que contém a senha e o email do linkedin fake

# acessando um arquivo em txt que contém o email e a senha do linkedin fake
login = open("credenciais.txt", encoding="utf-8") 

# Alocando a leitura do txt em uma váriavel e utilizando o método .readlines()
line = login.readlines() 

# Lendo a posicção[0] do meu txt que contém o email 
email = line[0]   

# Lendo a posição[1] do meu txt que contém o a senha 
senha = line[1] 

print('- Passo 1.2 concluído.')

In [ ]:
# 1.3 Buscando pelo campo de email e fazendo a inserção

# Mandando o selenium buscar pelo campo de inserção de email com o método find_element()
field_email = driver.find_element(By.ID,'username')
sleep(2)

# Mandando o selenium inserir o email com método send_keys()
field_email.send_keys(email)
sleep(2)

print('- Passo 1.3 concluído.')

In [3]:
#1.4 Buscando pelo campo de senha e fazendo a inserção

# Mandando o selenium bucar pelo campo de inserção da senha senha com o método find_element()
field_password = driver.find_element(By.NAME, 'session_password')
sleep(2)

# Mandando o selenium inserir a senha no campo de senha  com o método .send_keys()
field_password.send_keys(senha)
sleep(3)

print('- Passo 1.4 concluído.')

In [ ]:
# 1.5 Buscando pelo botão de login

# Mandando o selenium procurar pelo botão ENTRAR com o find_element(By.XPATH)
button_enter = driver.find_element(By.XPATH,'/html/body/div/main/div[2]/div[1]/form/div[3]/button')
sleep(2)

# Mandando o selenium clicar no botão entrar
button_enter.click()
sleep(2)

print('- Passo 1.5 concluído.')

In [ ]:
#Passo 02 usando o campo de busca dentro do linkedin

# Localizando o campo de busca via xpath
search_field = driver.find_element(By.XPATH,'/html/body/div[6]/header/div/div/div/div[1]/input')
sleep(3)

print('- Passo 02 concluído.')

In [ ]:
# 2.1 Inserindo os dados no campo de busca através do input
search_query = input('Qual perfil você quer fazer o scrape? ')
search_field.send_keys(search_query)
sleep(3)


In [4]:
# 2.2: Trazendo os resultados
search_field.send_keys(Keys.RETURN) #confirmando a inserção com Keys.RETURN que simula o botão enter do teclado

print('- Passo 2.2 concluído.')

Qual perfil você quer fazer o scrape?  engenheiro de dados pessoas


In [5]:
#Passo 03 Começando o scrape

# Definindo uma função que irá navegar entre os perfis e apendar os resultados em uma lista vazia
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'app-aware-link') 
    all_profile_URL = []
    
    # Laço for que irá percorrer os perfis e buscará pelo atributo href que é uma tag do html onde ficam os links    
    for profile in profiles:
        profile_URL = profile.get('href')

        # Condição que virifica se a lista ja possuí aquele perfil, caso contrário ela adiciona o perfil á lista        
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

print('- Passo 3.1 concluído')

In [6]:
# 3.1 definindo a quantidade de páginas que o BeaultifulSoup fará o scrape

# Input para inserir a quantidade de páginas que o programa irá percorrer
input_page = int(input('Em quantas páginas você quer fazer o scrape?: '))

URLs_all_page = []

# Laço for usado para percorrer a quantidade de páginas que o usuário solicitou via input
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    
    #Fazendo o papel de scroll do mouse indo ate o fim da página
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') 
    sleep(3)
    
    #localizando o botão de trocar de página via método By.CLASS_NAME
    next_button = driver.find_element(By.CLASS_NAME, "artdeco-pagination__button--next") 
    
    # Indicando a posição real do botão dentro da página do linkedin e avançando para a proxima página ex: 1, 2, 3 etc.
    driver.execute_script("arguments[0].click();", next_button)
    
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(3)

print('- Passo 3.1 concluído')

Em quantas páginas você quer fazer o scrape?:  1


- Passo 3.1 concluído


In [7]:
# Passo 4 gravando os resultados

# Gravando os dados em um aqruivo .CSV

with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Nome', 'Profissão', 'Localização', 'URL_Perfil']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Acessnado o perfil: ', linkedin_URL)
        sleep(3)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        try:
            name = info_div.find('li', class_='inline t-24 t-black t-normal break-words').get_text().strip() #Remove unnecessary characters 
            print('--- Profile name is: ', name)
            location = info_div.find('li', class_='t-16 t-black t-normal inline-block').get_text().strip() #Remove unnecessary characters 
            print('--- Profile location is: ', location)
            title = info_div.find('h2', class_='mt1 t-18 t-black t-normal break-words').get_text().strip()
            print('--- Profile title is: ', title)
            writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
            print('\n')
        except:
            pass

print('- Finalizado!')

- Acessnado o perfil:  https://www.linkedin.com/feed/?nis=true&
- Acessnado o perfil:  https://www.linkedin.com/mynetwork/?
- Acessnado o perfil:  https://www.linkedin.com/jobs/?
- Acessnado o perfil:  https://www.linkedin.com/messaging/?
- Acessnado o perfil:  https://www.linkedin.com/notifications/?
- Finalizado!
